- loglin plot with regional french data
- compare the FR numbers from here to those of Hopkins
- same for departement data
- pareil pour les "en réa"

In [101]:
import pandas as pd
import numpy as np
import yaml
import altair as alt
import seaborn as sns
import matplotlib.pyplot as plt

In [102]:
#file = ('./data/agences-regionales-sante/'
#       'auvergne-rhone-alpes/2020-02-27.yaml')
#fruits_list = yaml.load(open(file), Loader=yaml.FullLoader)

In [103]:
csv_filename = './data/dist/chiffres-cles.csv'
df = pd.read_csv(csv_filename)

In [104]:
df.date = pd.to_datetime(df.date, format='%Y-%m-%d')
dff = df.loc[(df.granularite=='region') & (df['source_nom'].str.contains("ARS")),
            ['date', 'maille_nom', 'deces', 'cas_confirmes', 'reanimation']]

In [105]:
dff[dff['maille_nom']=='Occitanie']

,date,maille_nom,deces,cas_confirmes,reanimation
758,2020-03-07,Occitanie,NaN,29.0,1.0
967,2020-03-09,Occitanie,NaN,60.0,2.0
1243,2020-03-11,Occitanie,NaN,118.0,8.0
1380,2020-03-12,Occitanie,NaN,150.0,11.0
1501,2020-03-13,Occitanie,NaN,175.0,17.0
1628,2020-03-14,Occitanie,NaN,210.0,17.0
1743,2020-03-15,Occitanie,1.0,254.0,23.0
1855,2020-03-16,Occitanie,3.0,284.0,23.0
1970,2020-03-17,Occitanie,5.0,313.0,32.0
2078,2020-03-18,Occitanie,10.0,389.0,37.0


In [106]:
#dff = dff.dropna() 
myregions = np.unique(dff.maille_nom)
dt_cols = np.unique(dff.date)

In [107]:
LAST_DATE = dff.date.max()
LAST_DATE

Timestamp('2020-03-22 00:00:00')

In [108]:
#countries = dff[dff['date'].eq(LAST_DATE) & dff['Confirmed Cases'].ge(MIN_CASES) & dff['Country'].ne('China')].sort_values(by='Confirmed Cases', ascending=False)
#countries = countries['Country'].values

## Opendatasoft

https://public.opendatasoft.com/explore/dataset/evolution-de-la-pandemie-de-covid-19-en-france/export/

In [115]:
# cas confirmés uniquement
# ods_dataset_id = 'evolution-de-la-pandemie-de-covid-19-en-france'
filename = 'evolution-de-la-pandemie-de-covid-19-en-france.csv'
df_ods_confirmes = pd.read_csv(filename, sep=';')

In [117]:
df_ods_confirmes['date'] = pd.to_datetime(df_ods_confirmes['Date'])
df_ods_confirmes['cas_confirmes'] = df_ods_confirmes['Nombre de cas confirmés']
df_ods_confirmes['maille_nom'] = df_ods_confirmes['Nom officiel']
df_ods_confirmes = df_ods_confirmes[['date','maille_nom', 'cas_confirmes']]
df_ods_confirmes = df_ods_confirmes.sort_values(by='date')

In [118]:
df_ods_confirmes[df_ods_confirmes.maille_nom=='Occitanie']

,date,maille_nom,cas_confirmes
208,2020-03-04,Occitanie,9
185,2020-03-05,Occitanie,11
209,2020-03-06,Occitanie,18
135,2020-03-07,Occitanie,36
218,2020-03-10,Occitanie,70
22,2020-03-11,Occitanie,87
191,2020-03-12,Occitanie,133
85,2020-03-13,Occitanie,172
202,2020-03-14,Occitanie,214
169,2020-03-15,Occitanie,243


## Analyse

In [119]:
#hide
MIN_CONFIRMES = 10
LAST_DATE = dt_cols[-1]

#myregion = dff[dff['date'].eq(LAST_DATE) & dff['deces'].ge(MIN_CASES)
#       ].sort_values(by='deces', ascending=False)
myregion = dff[dff['cas_confirmes'].ge(MIN_CONFIRMES)
       ].sort_values(by='cas_confirmes', ascending=False)
myregion = myregion['maille_nom'].values

In [120]:
myregion = np.unique(myregion)
myregion

array(['Auvergne-Rhône-Alpes', 'Bourgogne-Franche-Comté', 'Bretagne',
       'Centre-Val de Loire', 'Corse', 'Grand-Est', 'Guadeloupe',
       'Guyane', 'Hauts-de-France', 'Ile-de-France', 'Martinique',
       'Mayotte', 'Normandie', 'Nouvelle-Aquitaine', 'Occitanie',
       'Pays de la Loire', 'Provence-Alpes-Côte d’Azur'], dtype=object)

In [121]:
#hide
def get_country_colors(x):
    mapping = {
        'Italy': 'black',
        'Iran': '#A1BA59',
        'South Korea': '#A1BA59', #'#E45756',
        'Spain': '#E45756', #'#F58518',
        'Germany': '#9D755D',
        'France': '#2495D3', #'#F58518',
        'US': '#2495D3',
        'Switzerland': '#9D755D',
        'Norway': '#C1B7AD',
        'United Kingdom': '#C1B7AD', #'#2495D3',
        'Netherlands': '#F58518', #'#C1B7AD',
        'Sweden': '#C1B7AD',
        'Belgium': '#C1B7AD',
        'Denmark': '#C1B7AD',
        'Austria': '#C1B7AD',
        'Japan': '#9467bd'}
    return mapping.get(x, '#C1B7AD')

In [122]:
import random
get_colors = lambda n: list(map(lambda i: "#" + "%06x" % random.randint(0, 0xFFFFFF),range(n)))
color_range = get_colors(15) # sample return:  ['#8af5da', '#fbc08c', '#b741d0', '#e599f1', '#bbcb59', '#a2a6c0']

# Décès

In [263]:
#hide
SINCE_CASES_NUM = 2
COL_X = f'Days since {SINCE_CASES_NUM}th death'
dff2 = dff[dff['maille_nom'].isin(myregions)].copy()
days_since = (dff2.assign(F=dff2['deces'].ge(SINCE_CASES_NUM))
              .set_index('date')
              .groupby('maille_nom')['F'].transform('idxmax'))
dff2[COL_X] = (dff2['date'] - days_since.values).dt.days.values
dff2 = dff2[dff2[COL_X].ge(0)]

In [264]:
dff2[dff2['maille_nom']=='Grand-Est']

,date,maille_nom,deces,cas_confirmes,reanimation,Days since 2th death
861,2020-03-08,Grand-Est,3.0,262.0,45.0,0
964,2020-03-09,Grand-Est,4.0,310.0,NaN,1
1093,2020-03-10,Grand-Est,5.0,464.0,NaN,2
1236,2020-03-11,Grand-Est,9.0,587.0,NaN,3
1493,2020-03-13,Grand-Est,21.0,910.0,NaN,5
1620,2020-03-14,Grand-Est,24.0,1085.0,NaN,6
1735,2020-03-15,Grand-Est,45.0,1378.0,NaN,7
1847,2020-03-16,Grand-Est,51.0,1543.0,NaN,8
1962,2020-03-17,Grand-Est,61.0,1820.0,NaN,9
2183,2020-03-19,Grand-Est,93.0,NaN,300.0,11


In [265]:
# Need to remove 0 to get the log scale to not generate -inf
dff2.deces[dff2.deces==0]=np.nan

In [273]:
#hide_input
baseline_countries = ['Occitanie','Hauts-de-France','Ile-de-France','Grand-Est']
max_date = dff2['date'].max()
color_domain = list(dff2['maille_nom'].unique())
#color_range = list(map(get_country_colors, color_domain))

def make_since_chart(highlight_countries=[], baseline_countries=baseline_countries):
    selection = alt.selection_multi(fields=['maille_nom'], bind='legend', 
                                    init=[{'maille_nom': x} for x in highlight_countries + baseline_countries])

    base = alt.Chart(dff2, width=550).encode(
        x=f'{COL_X}:Q',
        y=alt.Y('deces:Q', scale=alt.Scale(type='log'), axis=alt.Axis(title='Cumulative Deaths')),
        color=alt.Color('maille_nom:N', scale=alt.Scale(domain=color_domain, range=color_range)),
        tooltip=list(dff2),
        opacity=alt.condition(selection, alt.value(1), alt.value(0.05))
    )
    max_day = dff2[COL_X].max()
    ref = pd.DataFrame([[x, SINCE_CASES_NUM*1.33**x] for x in range(max_day+1)], columns=[COL_X, 'deces'])
    base_ref = alt.Chart(ref).encode(x=f'{COL_X}:Q', y='deces:Q')
    return (
        alt.layer(
        base_ref.mark_line(color='black', opacity=.5, strokeDash=[3,3]),
        base_ref.transform_filter(
            alt.datum[COL_X] >= max_day
        ).mark_text(dy=-6, align='right', fontSize=10, text='33% Daily Growth'),
        base.mark_line(point=True).add_selection(selection),
        base.transform_filter(
            alt.datum['Date'] >= int(max_date.timestamp() * 1000)
        ).mark_text(dy=-8, align='right', fontWeight='bold').encode(text='maille_nom:N')
        )
    ).properties(
        #title=f"Compare {', '.join(highlight_countries)} death trajectory with {', '.join(baseline_countries)}"
        title=f"Compare countries death trajectories"
    )

In [274]:
#hide_input
chart = make_since_chart()
chart

alt.LayerChart(...)

# Confirmed cases

In [109]:
#hide
SINCE_CASES_NUM = 100
COL_X = f'Days since {SINCE_CASES_NUM}th case'
dff2 = dff[dff['maille_nom'].isin(myregions)].copy()
days_since = (dff2.assign(F=dff2['cas_confirmes'].ge(SINCE_CASES_NUM))
              .set_index('date')
              .groupby('maille_nom')['F'].transform('idxmax'))
dff2[COL_X] = (dff2['date'] - days_since.values).dt.days.values
dff2 = dff2[dff2[COL_X].ge(0)]

In [117]:
#dff2.deces = dff2.deces.astype('Int64')
#dff2.cas_confirmes = dff2.cas_confirmes.astype('Int64')
#dff2.reanimation = dff2.reanimation.astype('Int64')
dff2[dff2.maille_nom=='Ile-de-France']

,date,maille_nom,deces,cas_confirmes,reanimation,Days since 100th case
466,2020-03-06,Ile-de-France,1,131,<NA>,0
573,2020-03-07,Ile-de-France,1,161,<NA>,1
685,2020-03-08,Ile-de-France,1,226,<NA>,2
787,2020-03-09,Ile-de-France,1,283,<NA>,3
907,2020-03-10,Ile-de-France,4,283,<NA>,4
1048,2020-03-11,Ile-de-France,4,512,<NA>,5
1181,2020-03-12,Ile-de-France,4,605,<NA>,6
1300,2020-03-13,Ile-de-France,<NA>,876,<NA>,7
1424,2020-03-14,Ile-de-France,<NA>,1240,155,8
1758,2020-03-17,Ile-de-France,<NA>,1240,155,11


In [118]:
#hide_input
baseline_countries = ['Occitanie','Hauts-de-France','Ile-de-France']
max_date = dff2['date'].max()
color_domain = list(dff2['maille_nom'].unique())
#color_range = list(map(get_country_colors, color_domain))

def make_since_chart(highlight_countries=[], baseline_countries=baseline_countries):
    selection = alt.selection_multi(fields=['maille_nom'], bind='legend', 
                                    init=[{'maille_nom': x} for x in highlight_countries + baseline_countries])

    base = alt.Chart(dff2, width=550).encode(
        x=f'{COL_X}:Q',
        y=alt.Y('cas_confirmes:Q', scale=alt.Scale(type='linear'), axis=alt.Axis(title='Confirmed cases')),
        color=alt.Color('maille_nom:N', scale=alt.Scale(domain=color_domain, range=color_range)),
        tooltip=list(dff2),
        opacity=alt.condition(selection, alt.value(1), alt.value(0.05))
    )
    max_day = dff2[COL_X].max()
    ref = pd.DataFrame([[x, SINCE_CASES_NUM*1.33**x] for x in range(max_day+1)], columns=[COL_X, 'cas_confirmes'])
    base_ref = alt.Chart(ref).encode(x=f'{COL_X}:Q', y='cas_confirmes:Q')
    return (
        base_ref.mark_line(color='black', opacity=.5, strokeDash=[3,3]) +
        base_ref.transform_filter(
            alt.datum[COL_X] >= max_day
        ).mark_text(dy=-6, align='right', fontSize=10, text='33% Daily Growth') +
        base.mark_line(point=True).add_selection(selection) + 
        base.transform_filter(
            alt.datum['date'] >= int(max_date.timestamp() * 1000)
        ).mark_text(dy=-8, align='right', fontWeight='bold').encode(text='maille_nom:N')
    ).properties(
        #title=f"Compare {', '.join(highlight_countries)} death trajectory with {', '.join(baseline_countries)}"
        title=f"Compare countries confirmed case trajectories"
    )

chart = make_since_chart()
chart

TypeError: data type not understood

alt.LayerChart(...)